In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.12.0+cu116
True


In [2]:
import cv2
img = cv2.imread("dataset/panda.jpg")
cv2.namedWindow("winname",cv2.ACCESS_READ)  # 创建名字为winname的窗口
cv2.imshow("winname",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
from PIL import Image
img_path = "dataset/panda.jpg"
img2 = Image.open(img_path)
img2.show()
img2.size

(500, 375)

In [4]:
# 读取整个数据集
import os
dir_path = "dataset/birds_image"
img_path_list = os.listdir(dir_path)
print(img_path_list)
print(img_path_list[0])

['blue_robin.jpg', 'fenghu.jpg', 'huangli.jpg', 'robin.jpg', 'shanjiao.jpg']
blue_robin.jpg


In [5]:
from torch.utils.data import Dataset
help(Dataset)

Help on class Dataset in module torch.utils.data.dataset:

class Dataset(typing.Generic)
 |  Dataset(*args, **kwds)
 |  
 |  An abstract class representing a :class:`Dataset`.
 |  
 |  All datasets that represent a map from keys to data samples should subclass
 |  it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
 |  data sample for a given key. Subclasses could also optionally overwrite
 |  :meth:`__len__`, which is expected to return the size of the dataset by many
 |  :class:`~torch.utils.data.Sampler` implementations and the default options
 |  of :class:`~torch.utils.data.DataLoader`.
 |  
 |  .. note::
 |    :class:`~torch.utils.data.DataLoader` by default constructs a index
 |    sampler that yields integral indices.  To make it work with a map-style
 |    dataset with non-integral indices/keys, a custom sampler must be provided.
 |  
 |  Method resolution order:
 |      Dataset
 |      typing.Generic
 |      builtins.object
 |  
 |  Methods defined 

In [6]:
class MyDataset(Dataset):
    def __init__(self,  root_dir, label_dir):
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path = os.path.join(self.root_dir, self.label_dir)  # dataset/birds
        self.img_path = os.listdir(self.path)  # 图片名列表

    def __getitem__(self, idx):
        img_name = self.img_path[idx]
        img_item_path = os.path.join(self.root_dir, self.label_dir, img_name)
        img =Image.open(img_item_path)
        label = self.label_dir
        return img, label

    def __len__(self):
        return len(self.img_path)


root_dir = "dataset"
label_dir = "birds_image"
birds_dataset = MyDataset(root_dir, label_dir)
print(birds_dataset)


In [7]:
print(birds_dataset[0])
img, label = birds_dataset.__getitem__(0)
print(img, label)
print("=====================================================================================")
img, label = birds_dataset[0]
print(img, label)

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=670x447 at 0x1B5353F07C0>, 'birds_image')
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=670x447 at 0x1B5147D01F0> birds_image
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=670x447 at 0x1B515606DF0> birds_image


In [8]:
img, label = birds_dataset[0]
print(img, label)
img.show()

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=670x447 at 0x1B5353F0E20> birds_image


In [9]:
img, label = birds_dataset[1]
print(img, label)
img.show()

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=900x580 at 0x1B514B141F0> birds_image


In [10]:
cat_label_dir = "cats_image"
cats_dataset = MyDataset(root_dir, cat_label_dir)
print(cats_dataset)

In [11]:
img, label = cats_dataset[1]
img.show()

In [12]:
train_dataset = birds_dataset + cats_dataset
train_dataset

In [13]:
len(train_dataset)

9

In [14]:
len(birds_dataset)

5

In [15]:
len(cats_dataset)

4

In [16]:
img, label = train_dataset[4]
img.show()

In [17]:
img, label = train_dataset[5]
img.show()

TensorBoard

In [20]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("logs")

# writer.add_image()  # img_tensor (torch.Tensor, numpy.array, or string/blobname)
# y = 2x
for i in range(100):
    writer.add_scalar("y=2x", 2*i, i) 
    # scalar_value : 数值，对应y轴    global_step: 训练到多少步，对应x轴
    # 先是y，即2*i, 再是x

writer.close()

In [21]:
image_path = "dataset\panda.jpg"

In [22]:
from PIL import Image
img = Image.open(image_path)
print(type(img))

# PIL.JpegImage类型不满足上述img_tensor
# 可以利用numpy.array()对PIL图片进行转换
import numpy as np 
img_array = np.array(img)
print(type(img_array))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'numpy.ndarray'>


In [23]:
# 利用opencv读取可以得到numpy型
import cv2


In [24]:
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import numpy as np 

writer = SummaryWriter("logs")

image_path = "dataset/birds_image/fenghu.jpg"
img_PIL = Image.open(image_path)
#print(type(img))

img_array = np.array(img_PIL)
print(type(img_array))
print(img_array.shape)  # (375, 500, 3)   3通道在后面 


# 第二次运行需要修改 step 或者 title    此处从 1 改为2
writer.add_image("test", img_array, 2, dataformats = 'HWC')  # img_tensor (torch.Tensor, numpy.array, or string/blobname)
# 出现了维度问题，  img_tensor: 默认 is :math:`(3, H, W)`  3个通道
# 如果是 math:`(H, W, 3)`，可以使用 `dataformats` 去改    dataformats = 'HWC'(现在的shape形式)

writer.close()

<class 'numpy.ndarray'>
(580, 900, 3)


transforms的使用

In [25]:
from torchvision import transforms
from PIL import Image

img_path = "dataset/panda.jpg"
img = Image.open(img_path)
print(img)
print(type(img))

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x375 at 0x1B53585A130>
<class 'PIL.JpegImagePlugin.JpegImageFile'>


In [26]:
# transforms如何使用（python）
tensor_trans = transforms.ToTensor()
tensor_img = tensor_trans(img)

In [27]:
tensor_img

tensor([[[0.1412, 0.0941, 0.0902,  ..., 0.1020, 0.2980, 0.2706],
         [0.1137, 0.0627, 0.0627,  ..., 0.1961, 0.2549, 0.2431],
         [0.1373, 0.0824, 0.0784,  ..., 0.2196, 0.3255, 0.2275],
         ...,
         [0.2627, 0.2235, 0.2863,  ..., 0.1686, 0.1686, 0.1490],
         [0.2627, 0.2627, 0.4275,  ..., 0.2588, 0.1843, 0.1882],
         [0.4275, 0.5098, 0.4392,  ..., 0.1608, 0.1843, 0.1765]],

        [[0.0863, 0.0588, 0.0588,  ..., 0.1451, 0.2588, 0.3176],
         [0.0980, 0.0706, 0.0588,  ..., 0.1725, 0.2784, 0.2706],
         [0.0902, 0.0627, 0.0588,  ..., 0.1804, 0.2431, 0.3176],
         ...,
         [0.3098, 0.3137, 0.3529,  ..., 0.3255, 0.3333, 0.2471],
         [0.2824, 0.2745, 0.4078,  ..., 0.3922, 0.3451, 0.2196],
         [0.4431, 0.5020, 0.4353,  ..., 0.3294, 0.3020, 0.1922]],

        [[0.0824, 0.0706, 0.0510,  ..., 0.1294, 0.2549, 0.2784],
         [0.1020, 0.0275, 0.0392,  ..., 0.1882, 0.2706, 0.2431],
         [0.1059, 0.0392, 0.0431,  ..., 0.2118, 0.2706, 0.

In [28]:
# 为什么需要tensor数据类型

# 在神经网络中转换成tensor型，进行训练

import cv2
cv_img = cv2.imread(img_path)   # 读进来就是ndarray型的
print(cv_img)
print(cv_img.shape)
print(type(cv_img)) 


[[[ 21  22  36]
  [ 18  15  24]
  [ 13  15  23]
  ...
  [ 33  37  26]
  [ 65  66  76]
  [ 71  81  69]]

 [[ 26  25  29]
  [  7  18  16]
  [ 10  15  16]
  ...
  [ 48  44  50]
  [ 69  71  65]
  [ 62  69  62]]

 [[ 27  23  35]
  [ 10  16  21]
  [ 11  15  20]
  ...
  [ 54  46  56]
  [ 69  62  83]
  [ 71  81  58]]

 ...

 [[ 79  79  67]
  [ 88  80  57]
  [110  90  73]
  ...
  [ 57  83  43]
  [ 39  85  43]
  [ 24  63  38]]

 [[ 92  72  67]
  [105  70  67]
  [108 104 109]
  ...
  [ 83 100  66]
  [ 44  88  47]
  [ 33  56  48]]

 [[114 113 109]
  [142 128 130]
  [117 111 112]
  ...
  [ 77  84  41]
  [ 43  77  47]
  [ 35  49  45]]]
(375, 500, 3)
<class 'numpy.ndarray'>


In [29]:
from torch.utils.tensorboard import SummaryWriter

import cv2
cv_img = cv2.imread(img_path)   # 读进来就是ndarray型的

writer = SummaryWriter("logs")

tensor_trans = transforms.ToTensor()
tensor_img = tensor_trans(cv_img)

writer.add_image("Tensor_img",tensor_img)
writer.close()

常用Transforms

In [30]:
img = Image.open("dataset/birds_image/fenghu.jpg")
print(img)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=900x580 at 0x1B5147D01C0>


In [31]:
from torchvision import transforms

# __call__的作用
class Person:
    def __call__(self, name):
        print("__call__ " + "hello " + name)

    def hello(self, name):
        print("hello " + name)


person = Person()
person("TankLee")  # 内置方法
person.hello("TankLee")

__call__ hello TankLee
hello TankLee


In [32]:
from torch.utils.tensorboard import SummaryWriter

# ToTensor 使用
writer = SummaryWriter("logs")

img = cv2.imread("dataset/birds_image/fenghu.jpg")
trans_totensor = transforms.ToTensor()
img_tensor = trans_totensor(img)

writer.add_image("ToTensor",img_tensor)
writer.close()

In [33]:
# Normalize  归一化    给一个 tensor 型的img

writer = SummaryWriter("logs")

print(img_tensor[0][0][0])
trans_norm = transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # 均值和标准差
img_norm = trans_norm(img_tensor)
print(img_norm[0][0][0])

writer.add_image("Normalized",img_norm)

writer.close()

tensor(0.3608)
tensor(-0.2784)


In [34]:
writer = SummaryWriter("logs")

print(img_tensor[0][0][0])
trans_norm = transforms.Normalize([1, 3, 5], [3, 2, 1])  # 均值和标准差
img_norm = trans_norm(img_tensor)
print(img_norm[0][0][0])

writer.add_image("Normalized",img_norm, 2)

writer.close()

tensor(0.3608)
tensor(-0.2131)


In [35]:
# Risize 的使用
# 输入时 PIL_image

img = Image.open("dataset/cats_image/yinjian.jpg")
print(img.size)
trans_resize = transforms.Resize((512,512))
img_resize = trans_resize(img)
print(img_resize)


(1080, 812)
<PIL.Image.Image image mode=RGB size=512x512 at 0x1B5359B0520>


In [36]:
writer = SummaryWriter("logs")
# PIL -> resize -> img_resize PIL -> totensor -> img_resize tensor
img_resize = trans_totensor(img_resize)

writer.add_image("Resized image",img_resize, 0)

writer.close()

In [37]:
# compose 
writer = SummaryWriter("logs")

trans_resize_2 = transforms.Resize(512)
# PIL -> PIL -> tensor
trans_compose = transforms.Compose([trans_resize_2, trans_totensor])
img_resize_2 = trans_compose(img)
writer.add_image("Resized image",img_resize_2, 1)
print(img_resize_2)

writer.close()

tensor([[[0.5686, 0.5765, 0.5843,  ..., 0.5922, 0.5765, 0.5922],
         [0.5608, 0.5647, 0.5725,  ..., 0.6000, 0.5843, 0.5843],
         [0.5569, 0.5608, 0.5647,  ..., 0.6196, 0.5843, 0.5686],
         ...,
         [0.3686, 0.3725, 0.3686,  ..., 0.3647, 0.3608, 0.3647],
         [0.3608, 0.3686, 0.3647,  ..., 0.3647, 0.3569, 0.3647],
         [0.3725, 0.3765, 0.3647,  ..., 0.3647, 0.3490, 0.3529]],

        [[0.5647, 0.5725, 0.5804,  ..., 0.4902, 0.4745, 0.4902],
         [0.5569, 0.5608, 0.5686,  ..., 0.4980, 0.4824, 0.4824],
         [0.5529, 0.5569, 0.5608,  ..., 0.5176, 0.4824, 0.4667],
         ...,
         [0.3137, 0.3176, 0.3098,  ..., 0.1882, 0.1843, 0.1882],
         [0.3059, 0.3137, 0.3059,  ..., 0.1882, 0.1804, 0.1882],
         [0.3176, 0.3216, 0.3098,  ..., 0.1882, 0.1725, 0.1765]],

        [[0.4941, 0.5020, 0.5098,  ..., 0.3451, 0.3294, 0.3451],
         [0.4863, 0.4902, 0.4980,  ..., 0.3529, 0.3373, 0.3373],
         [0.4824, 0.4863, 0.4902,  ..., 0.3725, 0.3373, 0.

In [38]:
# RandomCrop  随机裁剪
writer = SummaryWriter("logs")

# trans_random = transforms.RandomCrop(512)
trans_random = transforms.RandomCrop((500,1000))
trans_compose_2 = transforms.Compose([trans_random, trans_totensor])
for i in range(10):
    img_crop = trans_compose_2(img)
    writer.add_image("RandomCrop",img_crop, i)

writer.close()


torchvision中数据集的使用

In [39]:
import torchvision

train_set = torchvision.datasets.CIFAR10(root="./dataset_CIFAR10", train = True, download= True)
test_set = torchvision.datasets.CIFAR10(root="./dataset_CIFAR10", train = False, download= True)

Files already downloaded and verified
Files already downloaded and verified


In [40]:
print(train_set[0])
img, target = train_set[0]
print("img: ", img )
print("target:", target)
print(train_set.classes[target])
img.show()

(<PIL.Image.Image image mode=RGB size=32x32 at 0x1B53884D6D0>, 6)
img:  <PIL.Image.Image image mode=RGB size=32x32 at 0x1B53884DA00>
target: 6
frog


In [41]:
train_set.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [42]:

dataset_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
train_set = torchvision.datasets.CIFAR10(root="./dataset_CIFAR10", transform = dataset_transform, train = True, download= True)
test_set = torchvision.datasets.CIFAR10(root="./dataset_CIFAR10", transform = dataset_transform, train = False, download= True)

print(train_set[0])

Files already downloaded and verified
Files already downloaded and verified
(tensor([[[0.2314, 0.1686, 0.1961,  ..., 0.6196, 0.5961, 0.5804],
         [0.0627, 0.0000, 0.0706,  ..., 0.4824, 0.4667, 0.4784],
         [0.0980, 0.0627, 0.1922,  ..., 0.4627, 0.4706, 0.4275],
         ...,
         [0.8157, 0.7882, 0.7765,  ..., 0.6275, 0.2196, 0.2078],
         [0.7059, 0.6784, 0.7294,  ..., 0.7216, 0.3804, 0.3255],
         [0.6941, 0.6588, 0.7020,  ..., 0.8471, 0.5922, 0.4824]],

        [[0.2431, 0.1804, 0.1882,  ..., 0.5176, 0.4902, 0.4863],
         [0.0784, 0.0000, 0.0314,  ..., 0.3451, 0.3255, 0.3412],
         [0.0941, 0.0275, 0.1059,  ..., 0.3294, 0.3294, 0.2863],
         ...,
         [0.6667, 0.6000, 0.6314,  ..., 0.5216, 0.1216, 0.1333],
         [0.5451, 0.4824, 0.5647,  ..., 0.5804, 0.2431, 0.2078],
         [0.5647, 0.5059, 0.5569,  ..., 0.7216, 0.4627, 0.3608]],

        [[0.2471, 0.1765, 0.1686,  ..., 0.4235, 0.4000, 0.4039],
         [0.0784, 0.0000, 0.0000,  ..., 0.2157

In [43]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("logs")
for i in range(10):
    img, target = train_set[i]
    writer.add_image("train_set",img, i)

writer.close()

DataLoader

In [44]:
import torchvision
from torch.utils.data import DataLoader

# 测试集
test_data = torchvision.datasets.CIFAR10("./dataset_CIFAR10", transform = torchvision.transforms.ToTensor(), train = False)

test_loader = DataLoader(test_data, batch_size= 4, shuffle=True, num_workers=0,drop_last=False)

# 测试数据集第一张图片
img,target = test_data[0]
print(img.shape)
print(target)
print(test_data.classes[target])

for data in test_loader:
    imgs, targets = data
    print(imgs.shape)
    print(targets)


torch.Size([3, 32, 32])
3
cat
torch.Size([4, 3, 32, 32])
tensor([7, 0, 2, 0])
torch.Size([4, 3, 32, 32])
tensor([6, 5, 9, 2])
torch.Size([4, 3, 32, 32])
tensor([7, 1, 0, 3])
torch.Size([4, 3, 32, 32])
tensor([3, 8, 5, 3])
torch.Size([4, 3, 32, 32])
tensor([2, 0, 1, 7])
torch.Size([4, 3, 32, 32])
tensor([3, 0, 3, 5])
torch.Size([4, 3, 32, 32])
tensor([5, 0, 2, 6])
torch.Size([4, 3, 32, 32])
tensor([1, 5, 9, 8])
torch.Size([4, 3, 32, 32])
tensor([9, 5, 5, 8])
torch.Size([4, 3, 32, 32])
tensor([9, 5, 9, 7])
torch.Size([4, 3, 32, 32])
tensor([9, 1, 3, 2])
torch.Size([4, 3, 32, 32])
tensor([3, 0, 0, 1])
torch.Size([4, 3, 32, 32])
tensor([6, 9, 3, 2])
torch.Size([4, 3, 32, 32])
tensor([0, 6, 2, 3])
torch.Size([4, 3, 32, 32])
tensor([0, 8, 4, 6])
torch.Size([4, 3, 32, 32])
tensor([4, 1, 5, 7])
torch.Size([4, 3, 32, 32])
tensor([8, 8, 1, 4])
torch.Size([4, 3, 32, 32])
tensor([2, 1, 9, 9])
torch.Size([4, 3, 32, 32])
tensor([8, 7, 6, 8])
torch.Size([4, 3, 32, 32])
tensor([2, 9, 3, 7])
torch.Size

In [45]:
from torch.utils.tensorboard import SummaryWriter

test_loader = DataLoader(test_data, batch_size= 64, shuffle=True, num_workers=0,drop_last=False)

writer = SummaryWriter("logs")
for epoch in range(2):
    step = 0
    for data in test_loader:
        imgs, targets = data
        writer.add_images("Epoch:{}".format(epoch), imgs, step)
        step = step + 1

writer.close()

# nn.module

In [46]:
import torch
from torch import nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()  # 必须要有的

    def forward(self,input):
        output = input + 1 
        return output

model = MyModel()
x = torch.tensor(1.0)
output = model(x)
print(output)

tensor(2.)


# conv2d


In [47]:
import torch
import torch.nn.functional as F

input = torch.tensor([[1,2,0,3,1],
                      [0,1,2,3,1],
                      [1,2,1,0,0],
                      [5,2,3,1,1],
                      [2,1,0,1,1]])
kernel = torch.tensor([[1,2,1],
                       [0,1,0],
                       [2,1,0]])

print("原始维度：")
print(input.shape)
print(kernel.shape)

# 需要变化尺寸
input= torch.reshape(input,(1,1,5,5))
kernel = torch.reshape(kernel,(1,1,3,3))

print("reshape后的维度：")
print(input.shape)
print(kernel.shape)

output = F.conv2d(input,kernel,stride = 1)
print("output:",output)
output2 = F.conv2d(input,kernel,stride = 2)
print("output2:",output2)
output3 = F.conv2d(input,kernel,stride = 1, padding= 1)
print("output3:",output3)


原始维度：
torch.Size([5, 5])
torch.Size([3, 3])
reshape后的维度：
torch.Size([1, 1, 5, 5])
torch.Size([1, 1, 3, 3])
output: tensor([[[[10, 12, 12],
          [18, 16, 16],
          [13,  9,  3]]]])
output2: tensor([[[[10, 12],
          [13,  3]]]])
output3: tensor([[[[ 1,  3,  4, 10,  8],
          [ 5, 10, 12, 12,  6],
          [ 7, 18, 16, 16,  8],
          [11, 13,  9,  3,  4],
          [14, 13,  9,  7,  4]]]])


卷积层 nn.Conv2d

In [48]:
import torch
import torchvision
import torch.nn as nn
from torch.nn import Conv2d
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("./dataset_CIFAR10",train = False, transform = torchvision.transforms.ToTensor(), download = True)
dataloader = DataLoader(dataset, batch_size = 64)

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2d(in_channels = 3, out_channels = 6, kernel_size = 3, padding = 0, stride = 1)
    
    def forward(self, x):
        x = self.conv1(x)
        return x

model = MyModel()
print(model)

writer = SummaryWriter("logs")
step = 0
for data in dataloader:
    imgs, targets = data
    output = model(imgs)
    print("imgs_shape:", imgs.shape)  # in_channels = 3
    print("output.shape:", output.shape) # batch_size = 64  out_channels = 6  卷积之后，原始图像减小为30
    writer.add_images("input", imgs, step)  # torch.Size([64, 3, 32, 32])
    
    # 6 个channel 无法显示，可以稍微改变一下让其显示  [64, 6, 30, 30]-> [xxx, 3, 30, 30]
    output = torch.reshape(output,(-1, 3, 30, 30))  # -1是自动识别，这里 xxx 是128，就是两批合到一起
    writer.add_images("output", output, step)  # torch.Size([64, 6, 30, 30])

    step = step + 1

writer.close()

Files already downloaded and verified
MyModel(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
)
imgs_shape: torch.Size([64, 3, 32, 32])
output.shape: torch.Size([64, 6, 30, 30])
imgs_shape: torch.Size([64, 3, 32, 32])
output.shape: torch.Size([64, 6, 30, 30])
imgs_shape: torch.Size([64, 3, 32, 32])
output.shape: torch.Size([64, 6, 30, 30])
imgs_shape: torch.Size([64, 3, 32, 32])
output.shape: torch.Size([64, 6, 30, 30])
imgs_shape: torch.Size([64, 3, 32, 32])
output.shape: torch.Size([64, 6, 30, 30])
imgs_shape: torch.Size([64, 3, 32, 32])
output.shape: torch.Size([64, 6, 30, 30])
imgs_shape: torch.Size([64, 3, 32, 32])
output.shape: torch.Size([64, 6, 30, 30])
imgs_shape: torch.Size([64, 3, 32, 32])
output.shape: torch.Size([64, 6, 30, 30])
imgs_shape: torch.Size([64, 3, 32, 32])
output.shape: torch.Size([64, 6, 30, 30])
imgs_shape: torch.Size([64, 3, 32, 32])
output.shape: torch.Size([64, 6, 30, 30])
imgs_shape: torch.Size([64, 3, 32, 32])
output.shape: torch.Size([64, 6,

# MaxPool2d

In [49]:
import torch
from torch import nn
from torch.nn import MaxPool2d

# "max_pool2d" not implemented for 'Long', 改成浮点型
input = torch.tensor([[1,2,0,3,1],
                      [0,1,2,3,1],
                      [1,2,1,0,0],
                      [5,2,3,1,1],
                      [2,1,0,1,1]], dtype = torch.float32)
input = torch.reshape(input,(-1, 1, 5, 5))
print("input.shape:", input.shape)

class Maxpool(nn.Module):
    def __init__(self):
        super(Maxpool,self).__init__()
        self.maxpool1 = MaxPool2d(kernel_size = 3, ceil_mode= True)  # ceil
    
    def forward(self, input):
        output = self.maxpool1(input)
        return output

maxpool = Maxpool()
output = maxpool(input)
print(output)



input.shape: torch.Size([1, 1, 5, 5])
tensor([[[[2., 3.],
          [5., 1.]]]])


In [50]:
import torch
from torch import nn
from torch.nn import MaxPool2d

# "max_pool2d" not implemented for 'Long', 改成浮点型
input = torch.tensor([[1,2,0,3,1],
                      [0,1,2,3,1],
                      [1,2,1,0,0],
                      [5,2,3,1,1],
                      [2,1,0,1,1]], dtype = torch.float32)
input = torch.reshape(input,(-1, 1, 5, 5))
print("input.shape:", input.shape)

class Maxpool(nn.Module):
    def __init__(self):
        super(Maxpool,self).__init__()
        self.maxpool1 = MaxPool2d(kernel_size = 3, ceil_mode= False)   # floor
    
    def forward(self, input):
        output = self.maxpool1(input)
        return output

maxpool2 = Maxpool()
output2 = maxpool2(input)
print(output2)


input.shape: torch.Size([1, 1, 5, 5])
tensor([[[[2.]]]])


In [51]:
import torch
import torchvision
from torch import nn
from torch.nn import MaxPool2d
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("./dataset_CIFAR10", train = False, download = True, transform = torchvision.transforms.ToTensor())

dataloader = DataLoader(dataset, batch_size = 64)

class Maxpool(nn.Module):
    def __init__(self):
        super(Maxpool,self).__init__()
        self.maxpool1 = MaxPool2d(kernel_size = 3, ceil_mode= False)   # floor
    
    def forward(self, input):
        output = self.maxpool1(input)
        return output

maxpool3 = Maxpool()

writer = SummaryWriter("logs")
step = 0

for data in dataloader:
    imgs, targets = data
    writer.add_images("input_maxpool", imgs, step)
    output3 = maxpool3(imgs)
    writer.add_images("output_maxpool", output3, step)
    step = step + 1

writer.close()

Files already downloaded and verified


# ReLU

In [52]:
import torch
from torch import nn 
from torch.nn import ReLU

input = torch.tensor([[1, -0.5], [-1, 3]])
input = torch.reshape(input, (-1,1,2,2))
print("input.shape = ", input.shape)

class Tank(nn.Module):
    def __init__(self):
        super(Tank,self).__init__()
        self.relu1 = ReLU()   # 默认inplace = False
    
    def forward(self, input):
        output = self.relu1(input)
        return output

tank = Tank()
output = tank(input)
print(output)

input.shape =  torch.Size([1, 1, 2, 2])
tensor([[[[1., 0.],
          [0., 3.]]]])


In [53]:
import torch
import torchvision
from torch import nn 
from torch.nn import ReLU, Sigmoid
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("dataset_CIFAR10", train = False, download =True, transform = torchvision.transforms.ToTensor())

dataloader = DataLoader(dataset, batch_size = 64)

class Tank(nn.Module):
    def __init__(self):
        super(Tank,self).__init__()
        self.relu1 = ReLU()   # 默认inplace = False
        self.sigmoid1 = Sigmoid()
    
    def forward(self, input):
        output = self.sigmoid1(input)
        return output

tank = Tank()

writer = SummaryWriter("logs")
step = 0

for data in dataloader:
    imgs, targets = data
    writer.add_images("input_sigmoid", imgs, step)
    output = tank(imgs)
    writer.add_images("output_sigmoid", output, step)
    step = step + 1

writer.close()

Files already downloaded and verified


# Linear  主要是网络后面的全连接层等等

In [56]:
import torch
import torchvision
from torch import nn 
from torch.nn import Linear
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("dataset_CIFAR10", train = False, download =True, transform = torchvision.transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size = 64)

class Tank(nn.Module):
    def __init__(self):
        super(Tank,self).__init__()
        self.linear1 = Linear(196608, 10)
    
    def forward(self, input):
        output = self.linear1(input)
        return output

tank = Tank()

for data in dataloader:
    imgs, targets = data
    print("imgs.shape:", imgs.shape)
    # output = torch.reshape(imgs,(1,1,1,-1))  # 先变成 1* 1 *n, 改用flatten
    output = torch.flatten(imgs)
    print ("output.shape:", output.shape)
    output2 = tank(output)
    print ("output_linear.shape:",output2.shape)


Files already downloaded and verified


RuntimeError: size mismatch, got 10, 10x196608,49152

# CIFAR10 model

In [57]:
from torch import nn
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear

class Tank(nn.Module):
    def __init__(self):
        super(Tank, self).__init__()
        self.conv1 = Conv2d(3,32,5,padding = 2)   # in_channels, out_channels, kernel_size, 需要根据网络计算padding
        self.maxpool1 = MaxPool2d(2)   # kernel_size
        self.conv2 = Conv2d(32,32,5,padding = 2) 
        self.maxpool2 = MaxPool2d(2)
        self.conv3 = Conv2d(32,64,5,padding = 2) 
        self.maxpool3 = MaxPool2d(2)
        self.flatten = Flatten()
        self.linear1 = Linear(1024, 64)
        self.linear2 = Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

tank = Tank()
print("网络结构：", tank)
# 测试网络是否正确
input = torch.ones((64, 3, 32, 32))  # batch_size = 64
output = tank(input)
print("output.shape:", output.shape)


网络结构： Tank(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=1024, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=10, bias=True)
)
output.shape: torch.Size([64, 10])


In [58]:
from torch import nn
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential

class Tank(nn.Module):
    def __init__(self):
        super(Tank, self).__init__()
        self.model = Sequential(
            Conv2d(3,32,5,padding = 2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding = 2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding = 2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x

tank = Tank()
print("网络结构：", tank)
# 测试网络是否正确
input = torch.ones((64, 3, 32, 32))  # batch_size = 64
output = tank(input)
print("output.shape:", output.shape)

网络结构： Tank(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)
output.shape: torch.Size([64, 10])


tensorboard可视化

In [59]:
from torch import nn
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential
from torch.utils.tensorboard import SummaryWriter


class Tank(nn.Module):
    def __init__(self):
        super(Tank, self).__init__()
        self.model = Sequential(
            Conv2d(3,32,5,padding = 2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding = 2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding = 2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x

tank = Tank()
print("网络结构：", tank)
# 测试网络是否正确
input = torch.ones((64, 3, 32, 32))  # batch_size = 64
output = tank(input)
print("output.shape:", output.shape)

writer = SummaryWriter("logs_model")
writer.add_graph(tank,input)
writer.close()

网络结构： Tank(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)
output.shape: torch.Size([64, 10])


Loss function

In [60]:
import torch
from torch.nn import L1Loss, MSELoss

inputs = torch.tensor([1,3,5], dtype= torch.float32 ) # 需要浮点数据
targets = torch.tensor([0,1,2], dtype= torch.float32)

inputs = torch.reshape(inputs,(1,1,1,3))
targets = torch.reshape(targets,(1,1,1,3))
print("inputs =",inputs)

loss = L1Loss(reduction = "mean")
result = loss(inputs,targets)
print("mean =",result)

loss = L1Loss(reduction = "sum")
result = loss(inputs,targets)
print("sum =",result)

loss_mse = MSELoss()
result = loss_mse(inputs,targets)
print("mseloss = ", result)

loss_mse = MSELoss(reduction = "sum")
result = loss_mse(inputs,targets)
print("mseloss_sum = ", result)


inputs = tensor([[[[1., 3., 5.]]]])
mean = tensor(2.)
sum = tensor(6.)
mseloss =  tensor(4.6667)
mseloss_sum =  tensor(14.)


In [61]:
from torch import nn
x = torch.tensor([0.1,0.2,0.3])  # 得分
y = torch.tensor([1])  # 命中的类别
x = torch.reshape(x, (1,3))  # 1 batchsize, 3 classes
loss_cross = nn.CrossEntropyLoss() 
result = loss_cross(x, y)
print("result =", result)

result = tensor(1.1019)


In [65]:
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential, CrossEntropyLoss

dataset = torchvision.datasets.CIFAR10("dataset_CIFAR10", train = False, download =True, transform = torchvision.transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size = 1)

class Tank(nn.Module):
    def __init__(self):
        super(Tank, self).__init__()
        self.model = Sequential(
            Conv2d(3,32,5,padding = 2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding = 2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding = 2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x

tank = Tank()
loss = CrossEntropyLoss()
for data in dataloader:
    imgs, targets = data
    outputs = tank(imgs)
    #print("outputs = ", outputs)
    #print("targets = ", targets)
    results_loss = loss(outputs, targets)
    #print("results_loss = ", results_loss)
    # 反向传播
    results_loss.backward()


Files already downloaded and verified


优化器

In [75]:
import torch
import time
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential, CrossEntropyLoss

dataset = torchvision.datasets.CIFAR10("dataset_CIFAR10", train = False, download =True, transform = torchvision.transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size = 1)

class Tank(nn.Module):
    def __init__(self):
        super(Tank, self).__init__()
        self.model = Sequential(
            Conv2d(3,32,5,padding = 2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding = 2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding = 2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x

tank = Tank()
loss = CrossEntropyLoss()
optimizer = torch.optim.SGD(tank.parameters(), lr = 0.01)

for epoch in range(5):   # epoch
    running_loss = 0.0 
    torch.cuda.synchronize()  # 计算训练时间
    start = time.time()
    for data in dataloader:
        imgs, targets = data
        outputs = tank(imgs)
        #print("outputs = ", outputs)
        #print("targets = ", targets)
        results_loss = loss(outputs, targets)
        
        optimizer.zero_grad()  # 梯度调为0
        results_loss.backward()  # 反向传播
        optimizer.step()  # 调优
        # print("results_loss = ", results_loss)
        running_loss += results_loss
    torch.cuda.synchronize()
    end = time.time()
    total_time = end - start
    print("running_loss =", running_loss, end = '  ')
    print('total_time:{:.2f}s'.format(total_time))



Files already downloaded and verified
running_loss = tensor(18690.8965, grad_fn=<AddBackward0>)  total_time:22.38s
running_loss = tensor(16122.2285, grad_fn=<AddBackward0>)  total_time:22.26s
running_loss = tensor(15493.0996, grad_fn=<AddBackward0>)  total_time:22.29s
running_loss = tensor(16044.8369, grad_fn=<AddBackward0>)  total_time:22.22s
running_loss = tensor(17855.9688, grad_fn=<AddBackward0>)  total_time:22.16s


现有网络训练

In [84]:
import torchvision
from torch import nn

# data_train = torchvision.datasets.ImageNet("ImageNet",split="train", download=True, transform = torchvision.transforms.ToTensor())

vgg = torchvision.models.vgg16(pretrained = False)
vgg16 = torchvision.models.vgg16(pretrained = True)
# print(vgg16)

dataset = torchvision.datasets.CIFAR10("dataset_CIFAR10", train = False, download =True, transform = torchvision.transforms.ToTensor())
# 增加一层，使1000类->10类
vgg16.classifier.add_module("add_linear", nn.Linear(1000,10))
# print(vgg16)

# 修改最后一层  4096 -> 10
# print(vgg)
vgg.classifier[6] = nn.Linear(4096,10)
print(vgg)

Files already downloaded and verified
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_s

模型训练

In [14]:
import torch
import torchvision
from torch import nn

vgg16 = torchvision.models.vgg16(pretrained = False)

# 保存方式 1 
torch.save(vgg16, "vgg16_method1.pth")  # 保存模型和参数

# 保存方式 2  (官方推荐)
torch.save(vgg16.state_dict(), "vgg16_method2.pth")  #保存参数

# 陷阱
class Tank(nn.Module):
    def __init__(self):
        super(Tank, self).__init__()
        self.conv = nn.Conv2d(3,64, 3)

    def forward(self, x):
        x = self.conv(x)
        return x

tank = Tank()
torch.save(tank, "tank_method1.pth") 


模型加载

In [15]:
import torch

# 加载方式1 -> 对应保存方式1
model = torch.load("vgg16_method1.pth")
# print(model)

# 加载方式2
model2 = torch.load("vgg16_method2.pth")
# print(model2)

# 陷阱 (写在同一个文件不会报错)
model3 = torch.load("tank_method1.pth")
print(model3)


Tank(
  (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
)


In [11]:
vgg16 = torchvision.models.vgg16(pretrained = False)
model = vgg16.load_state_dict(torch.load("vgg16_method2.pth"))
print(model)

d:\PYanaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
d:\PYanaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>
